**Processing airline data Group 10**

In [1]:
import matplotlib.pyplot as plt

#spark.conf.set("spark.sql.shuffle.partitions",60)
#spark.conf.set("spark.default.parallelism",60)
#spark.conf.set("spark.local.dir",'./spark_tmp')
# grabbing all csvs in raw folder just because
raw = spark.read.options(header=True,inferSchema=True).csv('airplane_raw')
raw.printSchema()
print(raw.take(1))




root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Carr

**How many unique airports are in our data?**

In [2]:

unique_airports = spark.read.options(header=True,inferSchema=True).csv('airplane_raw/metadata/airports.csv')
#print(unique_airports.count())
unique_airports.show()

actual_unique_airports = raw.select('Origin').union(raw.select('Dest')).distinct()
actual_unique_airports.show()
#print(actual_unique_airports.count())

+----+--------------------+------------------+-----+-------+-----------+------------+
|iata|             airport|              city|state|country|        lat|        long|
+----+--------------------+------------------+-----+-------+-----------+------------+
| 00M|            Thigpen |       Bay Springs|   MS|    USA|31.95376472|-89.23450472|
| 00R|Livingston Municipal|        Livingston|   TX|    USA|30.68586111|-95.01792778|
| 00V|         Meadow Lake|  Colorado Springs|   CO|    USA|38.94574889|-104.5698933|
| 01G|        Perry-Warsaw|             Perry|   NY|    USA|42.74134667|-78.05208056|
| 01J|    Hilliard Airpark|          Hilliard|   FL|    USA| 30.6880125|-81.90594389|
| 01M|   Tishomingo County|           Belmont|   MS|    USA|34.49166667|-88.20111111|
| 02A|         Gragg-Wade |           Clanton|   AL|    USA|32.85048667|-86.61145333|
| 02C|             Capitol|        Brookfield|   WI|    USA|   43.08751|-88.17786917|
| 02G|   Columbiana County|    East Liverpool|   OH|  

**How many records are flights that were cancelled vs those that were not cancelled?**

In [3]:
# simply counting total number and number of cancelled flights, the non-cancelled flights is the difference of these
'''
cancelled_count = raw.select('Cancelled').where('Cancelled = 1').count()
non_cancelled_count = raw.select('Cancelled').where('Cancelled= 0').count()
delayed_count = raw.select('Cancelled','DepDelay','ArrDelay').where('Cancelled == 1 OR DepDelay > 0 OR ArrDelay>0').count()
total_count = raw.count()
print("total flights: " + str(total_count))
print("cancelled flights: " + str(cancelled_count))
print("non-cancelled flights: " + str(non_cancelled_count))
print("expected non_cancelled: " + str(total_count - cancelled_count))
print("total cancelled and delayed flights: " + str(delayed_count))
'''

'\ncancelled_count = raw.select(\'Cancelled\').where(\'Cancelled = 1\').count()\nnon_cancelled_count = raw.select(\'Cancelled\').where(\'Cancelled= 0\').count()\ndelayed_count = raw.select(\'Cancelled\',\'DepDelay\',\'ArrDelay\').where(\'Cancelled == 1 OR DepDelay > 0 OR ArrDelay>0\').count()\ntotal_count = raw.count()\nprint("total flights: " + str(total_count))\nprint("cancelled flights: " + str(cancelled_count))\nprint("non-cancelled flights: " + str(non_cancelled_count))\nprint("expected non_cancelled: " + str(total_count - cancelled_count))\nprint("total cancelled and delayed flights: " + str(delayed_count))\n'

**Adding in weather data**

**Firstly, getting weather stations and longitude and latitude**

In [4]:
'''
IV. FORMAT OF "ghcnd-stations.txt"

------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------

'''
# we only want ID, latitude, longitude, and elevation, the other features do not matter to us
weather_stations = spark.read.options().text('weather_raw/metadata/ghcnd-stations.txt').rdd\
                                        .map(lambda x: x['value'])\
                                        .map(lambda x: [x[0:11],float(x[12:20]),float(x[21:30]),float(x[31:37])])\
                                        .toDF(['stationID','latitude','longitude','elevation'])
                                        
print(weather_stations.show())
#print(weather_stations.count())

+-----------+--------+---------+---------+
|  stationID|latitude|longitude|elevation|
+-----------+--------+---------+---------+
|ACW00011604| 17.1167| -61.7833|     10.1|
|ACW00011647| 17.1333| -61.7833|     19.2|
|AE000041196|  25.333|   55.517|     34.0|
|AEM00041194|  25.255|   55.364|     10.4|
|AEM00041217|  24.433|   54.651|     26.8|
|AEM00041218|  24.262|   55.609|    264.9|
|AF000040930|  35.317|   69.017|   3366.0|
|AFM00040938|   34.21|   62.228|    977.2|
|AFM00040948|  34.566|   69.212|   1791.3|
|AFM00040990|    31.5|    65.85|   1010.0|
|AG000060390| 36.7167|     3.25|     24.0|
|AG000060590| 30.5667|   2.8667|    397.0|
|AG000060611|   28.05|   9.6331|    561.0|
|AG000060680|    22.8|   5.4331|   1362.0|
|AGE00135039| 35.7297|     0.65|     50.0|
|AGE00147704|   36.97|     7.79|    161.0|
|AGE00147705|   36.78|     3.07|     59.0|
|AGE00147706|    36.8|     3.03|    344.0|
|AGE00147707|    36.8|     3.04|     38.0|
|AGE00147708|   36.72|     4.05|    222.0|
+----------

In [5]:
# matching airports used in dataset to their nearest weather station
actual_unique_airports_collect = actual_unique_airports.withColumnRenamed('Origin','iata')\
                                                        .join(unique_airports.select('iata','lat','long'),'iata')

# dont need elevation for haversine formula
weather_stations_collect = weather_stations.drop('elevation')
#print(weather_stations_collect.rdd.take(1))

**Matching airports to their closest weather station (so that we can fill records with relevant weather data)**

In [6]:
from math import radians, asin, sin,cos, sqrt
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
import heapq
import pickle
import os
# https://en.wikipedia.org/wiki/Haversine_formula
# can use haversine formula to compute distance based on longitude and latitude, assumes the Earth is a perfect sphere

# we want to map the airports used in our dataset to their closest weather stations, so that we can leverage weather data
# this haversine formula will return approximate distance in km between two points with longitude and latitude
def haversine_formula(long1,long2,lat1,lat2):
    long1 = radians(long1)
    long2 = radians(long2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    # radius of the earth as measured not from the poles
    radius = 6356.752
    return 2 * radius * asin( sqrt(\
                                       (sin((lat2-lat1)/2)**2) + \
                                       cos(lat1) * cos(lat2) * \
                                       (sin((long2-long1)/2)**2)\
                                  ) )
def filter_weather_stations(record):
    station_name = record[0]
    if not os.path.exists('./ghcnd_all/'+station_name+'.dly'):
        return False
    # getting the first line of the textfile and checking if the year <=1987
    with open('./ghcnd_all/'+station_name+'.dly') as f:
        first_line = f.readline()
        if int(first_line[11:15])<=1987:
            return True
    return False

# checking if we already created a pickle, if so, we do not need to do this costly computation
if os.path.exists('./airport_closest_stations.pickle'):
    print('pickle for airport to station mapping already generated!')
    with open('./airport_closest_stations.pickle', 'rb') as f:
            airport_nearest_weather_station = pickle.load(f)
    actual_unique_airports_collect.unpersist()
    weather_stations_collect.unpersist()
    del(actual_unique_airports_collect)
    del(weather_stations_collect)
    unique_airports.unpersist()
    del(unique_airports)
    actual_unique_airports.unpersist()
    del(actual_unique_airports)
else:
    actual_unique_airports_collect = actual_unique_airports_collect.collect()
    #weather_stations_collect = weather_stations_collect.filter(udf(lambda record: filter_weather_stations(record), BooleanType()))
    weather_stations_collect = weather_stations_collect.rdd\
                                                        .map(lambda x: (x['stationID'],x['latitude'],x['longitude']))\
                                                        .filter(filter_weather_stations).collect()
    # filtering out weather stations which havent been recording since 1987
    '''
    final_weather_stations = []
    count = 0
    for weather_station_record in weather_stations_collect:
        if os.path.exists('./ghcnd_all/'+weather_station_record[0]+'.dly'):
            data = spark.read.options().text('./ghcnd_all/'+weather_station_record[0]+'.dly')
            if data.rdd.map(lambda x:x['value']).map(lambda x: int(x[11:15])).sortBy(lambda x: x).take(1)[0] <= 1987:
                # lets add it to the final list
                final_weather_stations.append(weather_station_record)
        count+=1
        if (count%10000 ==0):
            print(count)

    del(weather_stations_collect)
    weather_stations_collect = final_weather_stations
    '''
    # creating a map from airport to the 5 nearest weather stations
    # if the size of max heap is 5, and the root is greater than the item in question, we can pop and insert the new item
    get_closest = 5
    airport_nearest_weather_station = {}
    for airport_record in actual_unique_airports_collect:
        max_heap=[]
        min_dist=None
        for weather_station_record in weather_stations_collect:
            long1 = airport_record[2]
            long2 = weather_station_record[2]
            lat1 = airport_record[1]
            lat2 = weather_station_record[1]
            dist = haversine_formula(long1,long2,lat1,lat2)
            if min_dist is None:
                min_dist = dist
            else:
                min_dist = min(min_dist,dist)
            if len(max_heap)<get_closest or abs(max_heap[0][0])>dist:
                # checking if the data starts from 1981
                #data = spark.read.options().text('./ghcnd_all/'+weather_station_record[0]+'.dly')
                #if (data.rdd.map(lambda x:x['value']).map(lambda x: int(x[11:15])).sortBy(lambda x: x).take(1)[0] <= 1982):
                # we are good then, lets add this station
                # pushing negative for max heap instead of min
                heapq.heappush(max_heap,(-min_dist,weather_station_record[0]))
                if len(max_heap)>get_closest:
                    heapq.heappop(max_heap)
        # inserting into map
        airport_nearest_weather_station[airport_record[0]] = [heap_element[1] for heap_element in max_heap]
        #print("matched airport to stations with min distance: " + str(min_dist)+" km")
        if min_dist > 10:
            print('got airport: ' + airport_record[0] + ' matched to min_distance ' + str(min_dist))


    # deleting collected arrays, we dont need them anymore
    del(weather_stations_collect)
    del(actual_unique_airports_collect)

    # saving the results of this to file, since it took a while to generate
    with open('./airport_closest_stations.pickle', 'wb') as f:
            pickle.dump(airport_nearest_weather_station, f, pickle.HIGHEST_PROTOCOL)


pickle for airport to station mapping already generated!


In [7]:
'''
Each ".dly" file contains data for one station.  The name of the file
corresponds to a station's identification code.  For example, "USC00026481.dly"
contains the data for the station with the identification code USC00026481).

Each record in a file contains one month of daily data.  The variables on each
line include the following:

------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
YEAR         12-15   Integer
MONTH        16-17   Integer
ELEMENT      18-21   Character
VALUE1       22-26   Integer
MFLAG1       27-27   Character
QFLAG1       28-28   Character
SFLAG1       29-29   Character
VALUE2       30-34   Integer
MFLAG2       35-35   Character
QFLAG2       36-36   Character
SFLAG2       37-37   Character
  .           .          .
  .           .          .
  .           .          .
VALUE31    262-266   Integer
MFLAG31    267-267   Character
QFLAG31    268-268   Character
SFLAG31    269-269   Character
------------------------------

These variables have the following definitions:

ID         is the station identification code.  Please see "ghcnd-stations.txt"
           for a complete list of stations and their metadata.
YEAR       is the year of the record.

MONTH      is the month of the record.

ELEMENT    is the element type.   There are five core elements as well as a number
           of addition elements.  
	   
	   The five core elements are:

        PRCP = Precipitation (tenths of mm)
   	   SNOW = Snowfall (mm)
	   SNWD = Snow depth (mm)
           TMAX = Maximum temperature (tenths of degrees C)
           TMIN = Minimum temperature (tenths of degrees C)
	   
	   The other elements are:
	   
	   ACMC = Average cloudiness midnight to midnight from 30-second 
	          ceilometer data (percent)
	   ACMH = Average cloudiness midnight to midnight from 
	          manual observations (percent)
           ACSC = Average cloudiness sunrise to sunset from 30-second 
	          ceilometer data (percent)
	   ACSH = Average cloudiness sunrise to sunset from manual 
	          observations (percent)
           ADPT = Average Dew Point Temperature for the day (tenths of degrees C)
           ASLP = Average Sea Level Pressure for the day (hPa * 10)
           ASTP = Average Station Level Pressure for the day (hPa * 10)
           AWBT = Average Wet Bulb Temperature for the day (tenths of degrees C)
           AWDR = Average daily wind direction (degrees)
	   AWND = Average daily wind speed (tenths of meters per second)
	   DAEV = Number of days included in the multiday evaporation
	          total (MDEV)
	   DAPR = Number of days included in the multiday precipiation 
	          total (MDPR)
           DASF = Number of days included in the multiday snowfall 
	          total (MDSF)		  
	   DATN = Number of days included in the multiday minimum temperature 
	         (MDTN)
	   DATX = Number of days included in the multiday maximum temperature 
	          (MDTX)
           DAWM = Number of days included in the multiday wind movement
	          (MDWM)
	   DWPR = Number of days with non-zero precipitation included in 
	          multiday precipitation total (MDPR)
	   EVAP = Evaporation of water from evaporation pan (tenths of mm)
	   FMTM = Time of fastest mile or fastest 1-minute wind 
	          (hours and minutes, i.e., HHMM)
	   FRGB = Base of frozen ground layer (cm)
	   FRGT = Top of frozen ground layer (cm)
	   FRTH = Thickness of frozen ground layer (cm)
	   GAHT = Difference between river and gauge height (cm)
	   MDEV = Multiday evaporation total (tenths of mm; use with DAEV)
	   MDPR = Multiday precipitation total (tenths of mm; use with DAPR and 
	          DWPR, if available)
	   MDSF = Multiday snowfall total 
	   MDTN = Multiday minimum temperature (tenths of degrees C; use with 
	          DATN)
	   MDTX = Multiday maximum temperature (tenths of degress C; use with 
	          DATX)
	   MDWM = Multiday wind movement (km)
           MNPN = Daily minimum temperature of water in an evaporation pan 
	         (tenths of degrees C)
           MXPN = Daily maximum temperature of water in an evaporation pan 
	         (tenths of degrees C)
	   PGTM = Peak gust time (hours and minutes, i.e., HHMM)
	   PSUN = Daily percent of possible sunshine (percent)
           RHAV = Average relative humidity for the day (percent)
           RHMN = Minimum relative humidity for the day (percent)
           RHMX = Maximum relative humidity for the day (percent)
	   SN*# = Minimum soil temperature (tenths of degrees C)
	          where * corresponds to a code
	          for ground cover and # corresponds to a code for soil 
		  depth.  
		  
		  Ground cover codes include the following:
		  0 = unknown
		  1 = grass
		  2 = fallow
		  3 = bare ground
		  4 = brome grass
		  5 = sod
		  6 = straw multch
		  7 = grass muck
		  8 = bare muck
		  
		  Depth codes include the following:
		  1 = 5 cm
		  2 = 10 cm
		  3 = 20 cm
		  4 = 50 cm
		  5 = 100 cm
		  6 = 150 cm
		  7 = 180 cm
		  
	   SX*# = Maximum soil temperature (tenths of degrees C) 
	          where * corresponds to a code for ground cover 
		  and # corresponds to a code for soil depth. 
		  See SN*# for ground cover and depth codes. 
           TAVG = Average temperature (tenths of degrees C)
	          [Note that TAVG from source 'S' corresponds
		   to an average for the period ending at
		   2400 UTC rather than local midnight]
           THIC = Thickness of ice on water (tenths of mm)	
 	   TOBS = Temperature at the time of observation (tenths of degrees C)
	   TSUN = Daily total sunshine (minutes)
	   WDF1 = Direction of fastest 1-minute wind (degrees)
	   WDF2 = Direction of fastest 2-minute wind (degrees)
	   WDF5 = Direction of fastest 5-second wind (degrees)
	   WDFG = Direction of peak wind gust (degrees)
	   WDFI = Direction of highest instantaneous wind (degrees)
	   WDFM = Fastest mile wind direction (degrees)
           WDMV = 24-hour wind movement (km)	   
           WESD = Water equivalent of snow on the ground (tenths of mm)
	   WESF = Water equivalent of snowfall (tenths of mm)
	   WSF1 = Fastest 1-minute wind speed (tenths of meters per second)
	   WSF2 = Fastest 2-minute wind speed (tenths of meters per second)
	   WSF5 = Fastest 5-second wind speed (tenths of meters per second)
	   WSFG = Peak gust wind speed (tenths of meters per second)
	   WSFI = Highest instantaneous wind speed (tenths of meters per second)
	   WSFM = Fastest mile wind speed (tenths of meters per second)
	   WT** = Weather Type where ** has one of the following values:
	   
                  01 = Fog, ice fog, or freezing fog (may include heavy fog)
                  02 = Heavy fog or heaving freezing fog (not always 
		       distinquished from fog)
                  03 = Thunder
                  04 = Ice pellets, sleet, snow pellets, or small hail 
                  05 = Hail (may include small hail)
                  06 = Glaze or rime 
                  07 = Dust, volcanic ash, blowing dust, blowing sand, or 
		       blowing obstruction
                  08 = Smoke or haze 
                  09 = Blowing or drifting snow
                  10 = Tornado, waterspout, or funnel cloud 
                  11 = High or damaging winds
                  12 = Blowing spray
                  13 = Mist
                  14 = Drizzle
                  15 = Freezing drizzle 
                  16 = Rain (may include freezing rain, drizzle, and
		       freezing drizzle) 
                  17 = Freezing rain 
                  18 = Snow, snow pellets, snow grains, or ice crystals
                  19 = Unknown source of precipitation 
                  21 = Ground fog 
                  22 = Ice fog or freezing fog
		  
            WV** = Weather in the Vicinity where ** has one of the following 
	           values:
		   
		   01 = Fog, ice fog, or freezing fog (may include heavy fog)
		   03 = Thunder
		   07 = Ash, dust, sand, or other blowing obstruction
		   18 = Snow or ice crystals
		   20 = Rain or snow shower
		   
VALUE1     is the value on the first day of the month (missing = -9999).

MFLAG1     is the measurement flag for the first day of the month.  There are
           ten possible values:

           Blank = no measurement information applicable
           B     = precipitation total formed from two 12-hour totals
           D     = precipitation total formed from four six-hour totals
	   H     = represents highest or lowest hourly temperature (TMAX or TMIN) 
	           or the average of hourly values (TAVG)
	   K     = converted from knots 
	   L     = temperature appears to be lagged with respect to reported
	           hour of observation 
           O     = converted from oktas 
	   P     = identified as "missing presumed zero" in DSI 3200 and 3206
           T     = trace of precipitation, snowfall, or snow depth
	   W     = converted from 16-point WBAN code (for wind direction)

QFLAG1     is the quality flag for the first day of the month.  There are 
           fourteen possible values:

           Blank = did not fail any quality assurance check
           D     = failed duplicate check
           G     = failed gap check
           I     = failed internal consistency check
           K     = failed streak/frequent-value check
	   L     = failed check on length of multiday period 
           M     = failed megaconsistency check
           N     = failed naught check
           O     = failed climatological outlier check
           R     = failed lagged range check
           S     = failed spatial consistency check
           T     = failed temporal consistency check
           W     = temperature too warm for snow
           X     = failed bounds check
	   Z     = flagged as a result of an official Datzilla 
	           investigation

SFLAG1     is the source flag for the first day of the month.  There are 
           thirty possible values (including blank, upper and 
	   lower case letters):

           Blank = No source (i.e., data value missing)
           0     = U.S. Cooperative Summary of the Day (NCDC DSI-3200)
           6     = CDMP Cooperative Summary of the Day (NCDC DSI-3206)
           7     = U.S. Cooperative Summary of the Day -- Transmitted 
	           via WxCoder3 (NCDC DSI-3207)
           A     = U.S. Automated Surface Observing System (ASOS) 
                   real-time data (since January 1, 2006)
	   a     = Australian data from the Australian Bureau of Meteorology
           B     = U.S. ASOS data for October 2000-December 2005 (NCDC 
                   DSI-3211)
	   b     = Belarus update
	   C     = Environment Canada
	   D     = Short time delay US National Weather Service CF6 daily 
	           summaries provided by the High Plains Regional Climate
		   Center
	   E     = European Climate Assessment and Dataset (Klein Tank 
	           et al., 2002)	   
           F     = U.S. Fort data 
           G     = Official Global Climate Observing System (GCOS) or 
                   other government-supplied data
           H     = High Plains Regional Climate Center real-time data
           I     = International collection (non U.S. data received through
	           personal contacts)
           K     = U.S. Cooperative Summary of the Day data digitized from
	           paper observer forms (from 2011 to present)
           M     = Monthly METAR Extract (additional ASOS data)
	   m     = Data from the Mexican National Water Commission (Comision
	           National del Agua -- CONAGUA)
	   N     = Community Collaborative Rain, Hail,and Snow (CoCoRaHS)
	   Q     = Data from several African countries that had been 
	           "quarantined", that is, withheld from public release
		   until permission was granted from the respective 
	           meteorological services
           R     = NCEI Reference Network Database (Climate Reference Network
	           and Regional Climate Reference Network)
	   r     = All-Russian Research Institute of Hydrometeorological 
	           Information-World Data Center
           S     = Global Summary of the Day (NCDC DSI-9618)
                   NOTE: "S" values are derived from hourly synoptic reports
                   exchanged on the Global Telecommunications System (GTS).
                   Daily values derived in this fashion may differ significantly
                   from "true" daily data, particularly for precipitation
                   (i.e., use with caution).
	   s     = China Meteorological Administration/National Meteorological Information Center/
	           Climatic Data Center (http://cdc.cma.gov.cn)
           T     = SNOwpack TELemtry (SNOTEL) data obtained from the U.S. 
	           Department of Agriculture's Natural Resources Conservation Service
	   U     = Remote Automatic Weather Station (RAWS) data obtained
	           from the Western Regional Climate Center	   
	   u     = Ukraine update	   
	   W     = WBAN/ASOS Summary of the Day from NCDC's Integrated 
	           Surface Data (ISD).  
           X     = U.S. First-Order Summary of the Day (NCDC DSI-3210)
	   Z     = Datzilla official additions or replacements 
	   z     = Uzbekistan update
	   
	   When data are available for the same time from more than one source,
	   the highest priority source is chosen according to the following
	   priority order (from highest to lowest):
	   Z,R,D,0,6,C,X,W,K,7,F,B,M,m,r,E,z,u,b,s,a,G,Q,I,A,N,T,U,H,S
	   
	   
VALUE2     is the value on the second day of the month

MFLAG2     is the measurement flag for the second day of the month.

QFLAG2     is the quality flag for the second day of the month.

SFLAG2     is the source flag for the second day of the month.

... and so on through the 31st day of the month.  Note: If the month has less 
than 31 days, then the remaining variables are set to missing (e.g., for April, 
VALUE31 = -9999, MFLAG31 = blank, QFLAG31 = blank, SFLAG31 = blank).

'''
print('blank cell')

blank cell


In [8]:
# now we have the nearest weather stations for each airport
#print(airport_nearest_weather_station['BGM'])
# this airport has nearest weather station USC00300684
# lets just see what the .dly file looks like
#weather_example = spark.read.options().text('./ghcnd_all/USC00300684.dly')
#print(weather_example.rdd.map(lambda x:x['value']).map(lambda x: int(x[11:15])).sortBy(lambda x: x).take(1))

#weather_example.unpersist()
#del(weather_example)

print('test cell')

# going through each record, finding the origin and destination year and day, and filling in weather info from the .dly files in ghcnd_all


test cell


**Joining weather data with our original data**

In [9]:
# basically for the values we are interested in, for origin and destination, we add the weather observation to the record
# getting the dataframe for only the weather records we might be interested in
interested_weather_stations = sc.parallelize(airport_nearest_weather_station.values()).flatMap(lambda x: x).distinct()
print(interested_weather_stations.take(2))
print(interested_weather_stations.count())

# mapping weather stations to their filenames for their weather records
interested_weather_stations = interested_weather_stations.map(lambda x: './ghcnd_all/'+x+'.dly')

interested_weather_data = spark.read.options().text(interested_weather_stations.collect())

interested_weather_stations.unpersist()
del(interested_weather_stations)

print(interested_weather_data.show())
print(interested_weather_data.count())



['USC00335750', 'USC00415701']
1712
+--------------------+
|               value|
+--------------------+
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190101...|
|USW00093822190102...|
|USW00093822190102...|
|USW00093822190102...|
|USW00093822190102...|
|USW00093822190102...|
|USW00093822190102...|
|USW00093822190102...|
|USW00093822190103...|
|USW00093822190103...|
|USW00093822190103...|
|USW00093822190103...|
+--------------------+
only showing top 20 rows

None
7380746


**Need to process the weather records to be usable**

In [10]:
 '''The five core elements are (ELEMENTS WE ARE INTERESTED IN FOR THE PROJECT!!!!!):

           PRCP = Precipitation (tenths of mm)
   	   SNOW = Snowfall (mm)
	   SNWD = Snow depth (mm)
           TMAX = Maximum temperature (tenths of degrees C)
           TMIN = Minimum temperature (tenths of degrees C)
	   
	   The other elements are:
	   # Maybe we can take one of these values if they are valid
	   ACMC = Average cloudiness midnight to midnight from 30-second 
	          ceilometer data (percent)
       ACMH = Average cloudiness midnight to midnight from 
	          manual observations (percent)
       ACSH = Average cloudiness sunrise to sunset from manual 
	          observations (percent)
       AWND = Average daily wind speed (tenths of meters per second)
       # similarly, pick one of these values if they are valid
       WSFM = Fastest mile wind speed (tenths of meters per second)
       WSFG = Peak gust wind speed (tenths of meters per second)
        
'''
print('blank cell')

blank cell


In [11]:
from pyspark.sql.functions import explode,collect_list,create_map,col,map_from_arrays

#raw.unpersist()
#del(raw)
# firstly filtering the dataset for elements that we want
interested_elements = {'PRCP','SNOW','SNWD','TMAX','TMIN','ACMC','ACMH','ACSH','AWND', 'WSFM', 'WSFG'}
#interested_elements = {'PRCP','SNOW','SNWD','TMAX','TMIN','AWND'}
def filter_weather(record):
    element = record[17:21]
    year = record[11:15]
    if element in interested_elements and int(year)>=1987:
        return True
    return False

num_interested_elements = len(interested_elements)

#print(interested_weather_data.rdd.map(lambda x: x['value']).filter(lambda x: x[17:21]=='TMAX' and int(x[11:15])==1987).take(1))
interested_weather_data = interested_weather_data.rdd.map(lambda x: x['value']).filter(filter_weather)
#print(interested_weather_data.count())

# need to process weather records to get basically, month,day,year and values for those days, we do not want -9999 values
def process_weather_record(record):
    station_id = record[0:11]
    year = int(record[11:15])
    month = int(record[15:17])
    element = record[17:21]
    day_values = {}
    offset = 21
    for i in range(31):
        measurement = int(record[offset+(i*8):offset+(i*8)+5])
        #print(measurement)
        if measurement != -9999:
            # we can add it to the dictionary
            day_values[i+1] = measurement
    return (station_id,year,month,element,day_values)


interested_weather_data = interested_weather_data.map(process_weather_record)
#print(interested_weather_data.map(lambda x: x[3]).take(1))
#print('do we have enough peak wind gust records? ' + str(interested_weather_data.map(lambda x: x[3])\
#                                                         .filter(lambda x: x=='WSFM' or x=='WSFG').count()))
#print(interested_weather_data.take(1))

#print(interested_weather_data.map(lambda x: ((x[0],x[1],x[2]),(x[3],x[4]))).groupByKey().map(lambda x: (x[0],list(x[1]))).take(1))
interested_weather = interested_weather_data.map(lambda x: ((x[0],x[1],x[2]),(x[3],x[4]))).groupByKey().map(lambda x: (x[0],list(x[1])))

def process_element_dictionaries(record):
    year = record[0][1]
    month = record[0][2]
    station = record[0][0]
    element_dictionary_entries = record[1]
    total_measurements={}
    for i in range(1,32):
        total_measurements[i] = {}
        #iterating through days
        for element, dictionary in element_dictionary_entries:
            if i in dictionary:
                # lets add this value to our day cumulative measurements
                total_measurements[i][element] = dictionary[i]
    return (station,year,month,total_measurements)


interested_weather = interested_weather.map(process_element_dictionaries)
#test = interested_weather.take(1)
#print(test)
#print(test[0][3].values())

# just seeing how many records contain all the data we are interested in? 
def complete_records_filter(record):
    for dictionary in record[3].values():
        if len(dictionary)!=0:
            # need this length check for example in 31st day of months with no 31st day
            element_set = dictionary.keys()
            if len(element_set) != num_interested_elements:
                # this stations is missing some features that we want
                return False
    return True


print('complete records:' + str(interested_weather.filter(complete_records_filter).count()))
# collecting as map
weather_station_data_map = interested_weather.map(lambda x: ((x[0],x[1],x[2]),x[3])).collectAsMap()

# now the goal is to accumulate elements for each (year,month,day) tuple for each station
'''
processed_weather_data = interested_weather_data.toDF(['station','year','month','element','measurements'])

processed_weather_data = processed_weather_data.select('station','year','month','element',explode('measurements')\
                                                      .alias('day','measurement'))\

del(interested_weather_data)
# need to combine all our elements together into a map
processed_weather_data = processed_weather_data.groupBy('station','year','month','day')\
                                               .agg(map_from_arrays(collect_list(processed_weather_data.element),collect_list(processed_weather_data.measurement)))\
                                               .alias('daily_measurements')
'''
#processed_weather_data = interested_weather.toDF(['station','year','month','daily_measurements'])
#processed_weather_data = processed_weather_data.select('station','year','month',explode('daily_measurements')\
                                                      #.alias('day','measurements'))

print(len(weather_station_data_map))
weather_station_data_map_broadcast = sc.broadcast(weather_station_data_map)

complete records:0
289325


**Joining weather elements to our airline dataset based on year, day, month, and closest station**


In [12]:
'''
Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- CarrierDelay: string (nullable = true)
 |-- WeatherDelay: string (nullable = true)
 |-- NASDelay: string (nullable = true)
 |-- SecurityDelay: string (nullable = true)
 |-- LateAircraftDelay: string (nullable = true)
'''
print('blank_cell')

blank_cell


In [13]:
from pyspark.sql import Row
from pyspark.sql.functions import monotonically_increasing_id
import copy


'''
IN CASE OOM
@F.udf(returnType = T.IntegerType())
def day_of_year(year,month,day):
    return date(year,month,day).timetuple().tm_yday

'''
    
def add_weather_to_airline_record(row):
    year = row['Year']
    month = row['Month']
    day = row['DayofMonth']
    airport = row['Airport']
    nearest_stations = airport_nearest_weather_station[airport]
    airport_weather = {}
    for station in nearest_stations:
        if (station,year,month) in weather_station_data_map_broadcast.value:
            weather_conditions = weather_station_data_map_broadcast.value[(station,year,month)][day]
            for key in weather_conditions:
                if key not in airport_weather or airport_weather[key] == -9999:
                    airport_weather[key] = weather_conditions[key]
    updated_row = row.asDict()
    for key in interested_elements:
        if key in airport_weather:
            updated_row[key] = airport_weather[key]
        else:
            updated_row[key] = -9999
    return Row(**updated_row)

def filter_bad_rows(row):
    values = row(0)
    for val in values:
        if type(val)=='int' and val == -9999:
            return False
    return True

# getting unique airport,year,month,day tuples that we need to map to weather
airport_instances = raw.select('Year','Month','DayofMonth',col('Origin').alias('Airport'))
dest_airport_instances = raw.select('Year','Month','DayofMonth',col('Dest').alias('Airport'))
final_airport_instances = airport_instances.union(dest_airport_instances).distinct().withColumn('id',monotonically_increasing_id())

# we can save final_airport instances to parquet
print(final_airport_instances.show())
print(final_airport_instances.count())

def process_only_id(row):
    new_dict = {}
    new_dict['id'] = row['id']
    for key in interested_elements:
        new_dict[key] = row[key]
    return Row(**new_dict)

# there are some airport codes that are in the data, but not in the metadata, we will drop those for sake of processing
added_weather_data = final_airport_instances.rdd.filter(lambda x: x['Airport'] in airport_nearest_weather_station)\
                                                .map(add_weather_to_airline_record)\
                                                .map(process_only_id).toDF()


print(added_weather_data.show())

final_airport_instances.coalesce(1).write.parquet('./airport_date_to_id_mapping')
added_weather_data.coalesce(1).write.parquet('./id_to_weather_mapping')


# basically, we make a mapping from (airport,year,month,day) to id
# then we make a mapping from id to weather data -> thus saving a bit of space
# based on the idea that many flights will take off from a given airport in a day and that many will land in a day
# so, no need to duplicate the data and join with the original dataset
# when we are processing the data in batches, we can just call our lookup then

# saving our mapping from (airport,year,month,day) to id in disk
# saving our mapping from id to weather in disk



print('wrote processed weather dataframe to parquet partitioned by year!')

+----+-----+----------+-------+---+
|Year|Month|DayofMonth|Airport| id|
+----+-----+----------+-------+---+
|1993|    1|         5|    TPA|  0|
|1993|    1|        22|    MCI|  1|
|1993|    1|        17|    OAJ|  2|
|1993|    1|        22|    SRQ|  3|
|1993|    1|        27|    ELM|  4|
|1993|    1|        14|    CAE|  5|
|1993|    1|        15|    ABQ|  6|
|1993|    1|        24|    MOB|  7|
|1993|    1|        30|    ORH|  8|
|1993|    1|        26|    SAV|  9|
|1993|    1|        18|    MKE| 10|
|1993|    1|        11|    MKE| 11|
|1993|    1|        27|    IAD| 12|
|1993|    1|        15|    BHM| 13|
|1993|    1|         1|    SMF| 14|
|1993|    1|        26|    SAT| 15|
|1993|    1|         5|    ACY| 16|
|1993|    1|         9|    TRI| 17|
|1993|    1|        17|    AMA| 18|
|1993|    1|         4|    OKC| 19|
+----+-----+----------+-------+---+
only showing top 20 rows

None
1708462
+---+-----+-----+-----+----+----+----+----+-----+-----+-----+----+
| id| ACSH| WSFM| WSFG|SNOW|TM

In [14]:
# how many records have average wind speed != -9999

# collecting tuple->id as map
#test_tuple_map = final_airport_instances.rdd.map(lambda x: ((x['Year'],x['Month'],x['DayofMonth'],x['Airport']),x['id'])).collectAsMap()

#test_id_wather_map = added_weather_data.rdd.map(lambda x: (x['id'],\
#                                                          (x['ACSH'],x['WSFM'],x['WSFG'],x['SNOW'],x['TMAX'],x['SNWD'],\
#                                                          x['PRCP'],x['AWND'],x['ACMH'],x['ACMC'],x['TMIN'])\
#                                                          )).collectAsMap()

**We have the weather data joined now!**

**What are the busiest airports that people fly out of?**

In [15]:
'''
# simply counting records for origin airport (doesnt matter if these flights were cancelled or not, its about intent)

# need to read in metadata for airport names
airport_metadata = spark.read.options(inferSchema=True,header=True).csv('airplane_raw/metadata/airports.csv')
iata_airport_pairs = airport_metadata.select('iata', 'airport')
origin_top_10 = raw.select('Origin').withColumnRenamed('Origin','iata')\
                                    .groupBy('iata')\
                                    .count()\
                                    .join(iata_airport_pairs,'iata','inner')\
                                    .orderBy('count',ascending=False)
origin_top_10.show()
'''
print('blank cell')

blank cell


**What are the busiest airports that people fly into?**

In [16]:
'''# simply counting records for destination airport (doesnt matter if these flights were cancelled or not, its about intent)
dest_top_10 = raw.select('Dest').withColumnRenamed('Dest','iata')\
                                    .groupBy('iata')\
                                    .count()\
                                    .join(iata_airport_pairs,'iata','inner')\
                                    .orderBy('count',ascending=False)
dest_top_10.show()
'''
print('blank cell')

blank cell


**What are the busiest seasons?**

In [17]:
'''
from pyspark.sql import functions as F
from pyspark.sql import types as T



# September through November is Fall, December through February is Winter, March through May is Spring, June through August is Summer

    


@F.udf(returnType=T.StringType())
def mapMonthToSeason(month):
    if 9<=month<=11:
        return "fall"
    elif month == 12 or month == 1 or month ==2:
        return "winter"
    elif 3<=month<=5:
        return "spring"
    else:
        return "summer"



# mapping flight records to their season
season_dataframe = raw.select('Month').withColumn('Month',mapMonthToSeason(F.col('Month'))).withColumnRenamed('Month','Season')\
                                      .groupBy('Season')\
                                      .count()\
                                      .show()'''
print('blank cell')

blank cell


**Which airport as the origin has the most delays?**

**Which airport as the destination has the most delays?**

**Which Seasons have the most delays?**

**Feature Exploration and Distribution Analysis**